# Load Libraries

In [1]:
!pip install ..

Processing /tf
  Preparing metadata (setup.py) ... done
  Created wheel for src: filename=src-0.0.1-py3-none-any.whl size=14284 sha256=d5a69f9e209623303aa0343d864b64af098f242015d6e7d0046e9690970c80c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-akoouvby/wheels/b8/4c/5d/041c4fc7e6c2e6d5dbdf9e9296283834aead4dadd6ecfdb44e
Successfully built src
  Attempting uninstall: src
    Found existing installation: src 0.0.1
    Uninstalling src-0.0.1:
      Successfully uninstalled src-0.0.1


In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import joblib
import tensorflow as tf
from tensorflow import keras 
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from keras.layers.core import Dense, Dropout, Activation
import keras.callbacks as callbacks
from keras.utils import np_utils


from src.functions.AuxiliarFunctions import *

# Load Data

In [3]:
df_config = pd.read_csv('../data/config.csv')
train_id = 1
df_config

,hash_id,label,wav_files_path,processed_file_folder,processed_file_path,wav_files_info,cv_alg,cv_folds,cv_path,preproc_alg,...,pipeline_path,scaler_alg,train_data_path,train_trgt_path,target_label_file,target_id_file,model_path,model_inits,model_neurons,model_status
0,-7182709064797668123,ShipsEar NN Classification,../data/shipsEar_AUDIOS,../data,../data/-7182709064797668123_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,Lofar,...,../data/pipelines,StandardScaler,../data/-7182709064797668123_train_data.csv,../data/-7182709064797668123_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/-7182709064797668123_hidden_neu...,../data/models/-7182709064797668123_model_stat...
1,1518950927628997345,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/1518950927628997345_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/pipelines,StandardScaler,../data/1518950927628997345_train_data.csv,../data/1518950927628997345_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/1518950927628997345_hidden_neur...,../data/models/1518950927628997345_model_statu...


In [4]:
df_train = pd.read_csv(df_config['train_data_path'][train_id])

In [5]:
df_train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,-0.809787,1.140782,0.552740,1.942452,-0.099175,0.122905,0.275852,1.482409,0.423330,1.247347,...,-0.699771,0.220017,-0.916476,0.515953,0.282666,0.945631,0.277907,1.574058,0.042095,1
1,2.582713,3.161590,0.083602,-2.986270,1.151863,2.075501,-0.596150,-0.524811,1.258575,-1.426467,...,-0.218064,0.154101,0.137039,-1.850546,2.839435,0.895683,0.104921,2.725624,-0.170036,1
2,3.388922,-0.312243,0.332491,-1.109370,0.389026,-0.028511,0.292879,-3.008659,-0.940198,-2.708882,...,-0.232607,-1.320325,-0.563335,0.552087,1.852921,-0.539202,0.366420,0.859907,-0.567231,0
3,2.299062,1.340423,-0.705663,0.243180,-0.178422,-2.069673,-0.533815,-2.986325,-0.977363,-1.303734,...,-1.848854,-1.474998,-1.686704,2.852161,3.344909,-0.228653,0.191883,0.918660,-0.124210,0
4,-1.559000,-0.551521,2.361235,-2.182949,-0.161701,-0.329970,0.588126,-2.942822,0.375951,-1.165616,...,1.030261,-2.005482,0.872376,-1.896221,2.920265,1.582279,1.642354,0.126912,0.468424,0


## Fit a preprocessing pipeline for each kFold

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

index_path = df_config['cv_path'][0]
pipe_path = df_config['pipeline_path'][0]
n_folds =  df_config['cv_folds'][0]

data = df_train.drop(columns=['target'])

for ifold in range(n_folds):
    index_file = '%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                       ifold, n_folds)
    with open(os.path.join(index_path,index_file),'rb') as file_handler:
            [trn_idx,val_idx] = pickle.load(file_handler)
    
    # criando o pipeline
    scaler = None
    if df_config['scaler_alg'][train_id] == "StandardScaler":
        scaler = StandardScaler()
    elif df_config['scaler_alg'][train_id] == "MinMaxScaler":
        scaler = MinMaxScaler()
    
    pipe = Pipeline(steps=[("scaler", scaler)])
    pipe.fit(data.loc[trn_idx,:])
    
    pipe_name = '%s_CV_fold_%i_of_%i_cv_pipe.pkl'%(df_config['hash_id'][train_id],
                                                   ifold, n_folds)
    with open(os.path.join(pipe_path,pipe_name),'wb') as file_handler:
        joblib.dump(pipe, file_handler)

# Define Model Class

In [7]:
class MLPModel:
    def __init__(self, n_hidden_neurons=2, verbose=2):
        self.n_hidden_neurons = n_hidden_neurons
        self.model = None
        self.trn_history = None
        self.trained = False
        self.verbose = verbose
    def __str__(self):
        m_str = 'Class MLPModel\n'
        if self.trained:
            m_str += 'Model is fitted, '
        else:
            m_str += 'Model is not fitted, '
        m_str += 'instance created with %i hidden neurons'%(self.n_hidden_neurons) 
        return m_str
    def create_model(self, data, target, random_state=0, learning_rate=0.01):
        #tf.random.set_seed(random_state)

        model = tf.keras.Sequential()
        
        # add a input to isolate the input of NN model
        model.add(tf.keras.Input(shape=(data.shape[1],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=self.n_hidden_neurons,
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                                    bias_regularizer=regularizers.L2(1e-4),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(hidden_layer)
        # add a non-linear output layer with max sparse target shape
        output_layer = layers.Dense(units=target.shape[1],
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(output_layer)
        # creating a optimization function using steepest gradient
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                                  decay_steps=100,
                                                                  decay_rate=0.9)
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

        # compile the model
        loss = keras.losses.CategoricalCrossentropy(from_logits=False,
                                                    label_smoothing=0.0,
                                                    axis=-1,
                                                    reduction="auto",
                                                    name="cat_crossent",)


        cat_cross = keras.losses.BinaryCrossentropy(from_logits=False,
                                                         label_smoothing=0.0,
                                                         axis=-1,
                                                         reduction="auto",
                                                         name="cat_crossent_met",)
        cat_acc_metric = keras.metrics.BinaryAccuracy(name="cat_acc", dtype=None)
        acc_metric = keras.metrics.Accuracy(name="accuracy",dtype=None)
        mse_metric = keras.metrics.MeanSquaredError(name="mse", dtype=None)
        rmse_metric = keras.metrics.RootMeanSquaredError(name="rmse", dtype=None)

        model.compile(loss="mean_squared_error", 
                      optimizer=optimizer,
                      metrics=[acc_metric,mse_metric,rmse_metric])
        return model
    def fit(self, X, Y,
            trn_id=None, 
            val_id=None, 
            epochs=50,
            batch_size=4,
            patience = 100,
            learning_rate=0.01, random_state=0):
        
        X_copy = X.copy()
        Y_copy = Y.copy()
        
        model = self.create_model(X_copy,Y_copy, random_state=random_state, learning_rate=learning_rate)
        
        # early stopping to avoid overtraining
        earlyStopping = callbacks.EarlyStopping(monitor='val_loss', 
                                                patience=patience,verbose=self.verbose, 
                                                mode='auto')
    
        trn_desc = model.fit(X_copy[trn_id,:], Y_copy[trn_id],
                             epochs=epochs,
                             batch_size=batch_size,
                             callbacks=[earlyStopping], 
                             verbose=self.verbose,
                             validation_data=(X_copy[val_id,:],
                                              Y_copy[val_id]),
                            )
        self.model = model
        self.trn_history = trn_desc
        self.trained = True
    def predict(self, data):
        return self.model.predict(data)
    def save(self, file_path):
        with open(file_path,'wb') as file_handler:
            joblib.dump([self.n_hidden_neurons, self.model,
                        self.trn_history, self.trained], file_handler)
    def load(self, file_path):
        with open(file_path,'rb') as file_handler:
            [self.n_hidden_neurons, self.model, self.trn_history, self.trained]= joblib.load(file_handler)
    def model_with_no_output_layer(self):
        buffer_model = tf.keras.Sequential()    
        # add a input to isolate the input of NN model
        buffer_model.add(tf.keras.Input(shape=(model.model.layers[0].get_weights()[0].shape[0],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=model.model.layers[0].get_weights()[1].shape[0],
                                    activation='tanh')
        buffer_model.add(hidden_layer)    
        output_layer = layers.Dense(units=1,activation='tanh')
    
        for idx, layer in enumerate(buffer_model.layers):
            layer.set_weights(model.model.layers[idx].get_weights())
        return buffer_model
    def predict_one_layer_before_output(self, data):
        buffer_model = self.model_with_no_output_layer()
        return buffer_model.predict(data)


# kFold training

In [8]:
def get_train_description(df_config, train_id):
    str_out = '\n\n\n'
    str_out +=  '=======================================\n'
    str_out +=  '%s Training Process'%(df_config['label'][train_id])
    str_out += '\n'
    str_out +=  '=======================================\n'
    
    str_out += 'Processing %s'%(df_config['train_data_path'][train_id])+'\n'
    str_out += 'Hidden Neurons:'
    hidden_neurons = ' '
    for ihidden_neuron in list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id])):
        hidden_neurons += str(ihidden_neuron)+', '
    str_out += hidden_neurons[:-2]
    str_out += '\n'
    str_out += 'CV Folds: %s\n'%(df_config['cv_folds'][train_id])
    str_out += 'Inits: %s\n'%(df_config['model_inits'][train_id])
    return str_out

In [32]:
class MLPModel:
    def __init__(self, n_hidden_neurons=2, verbose=2):
        self.n_hidden_neurons = n_hidden_neurons
        self.model = None
        self.trn_history = None
        self.trained = False
        self.verbose = verbose
    def __str__(self):
        m_str = 'Class MLPModel\n'
        if self.trained:
            m_str += 'Model is fitted, '
        else:
            m_str += 'Model is not fitted, '
        m_str += 'instance created with %i hidden neurons'%(self.n_hidden_neurons) 
        return m_str
    def loss_function(self):
        loss = tf.keras.losses.BinaryCrossentropy(
            from_logits=False,
            label_smoothing=0.0,
            axis=-1,
            reduction="auto",
            name="binary_crossentropy",
        )
        return loss
        
    def create_model(self, data, target, random_state=0, learning_rate=0.01):
        #tf.random.set_seed(random_state)

        model = tf.keras.Sequential()
        
        # add a input to isolate the input of NN model
        model.add(tf.keras.Input(shape=(data.shape[1],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=self.n_hidden_neurons,
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                                    bias_regularizer=regularizers.L2(1e-4),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(hidden_layer)
        # add a non-linear output layer with max sparse target shape
        output_layer = layers.Dense(units=target.shape[1],
                                    activation='tanh',
                                    kernel_initializer=initializers.RandomNormal(stddev=0.01),
                                    bias_initializer=initializers.Zeros()
                                   )
        model.add(output_layer)
        # creating a optimization function using steepest gradient
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=learning_rate,
                                                                  decay_steps=100,
                                                                  decay_rate=0.9)
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

        optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False,
                                amsgrad=False, weight_decay=None, clipnorm=None,
                                clipvalue=None,global_clipnorm=None,use_ema=False,
                                ema_momentum=0.99,ema_overwrite_frequency=None,
                                jit_compile=True,name="SGD",)
        
        
        # compile the model
        loss = keras.losses.CategoricalCrossentropy(from_logits=True,
                                                    label_smoothing=0.0,
                                                    axis=-1,
                                                    reduction="auto",
                                                    name="cat_crossent",)

        loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
       
        cat_acc_metric = keras.metrics.CategoricalAccuracy(name="cat_acc", dtype=None)
        acc_metric = keras.metrics.Accuracy(name="accuracy",dtype=None)
        mse_metric = keras.metrics.MeanSquaredError(name="mse", dtype=None)
        rmse_metric = keras.metrics.RootMeanSquaredError(name="rmse", dtype=None)

        model.compile(loss=loss, 
                      optimizer=optimizer,
                      metrics=[cat_acc_metric,acc_metric,mse_metric,rmse_metric])
        return model
    def fit(self, X, Y,
            trn_id=None, 
            val_id=None, 
            epochs=50,
            batch_size=4,
            patience = 100,
            learning_rate=0.01, random_state=0):
        
        X_copy = X.copy()
        Y_copy = Y.copy()
        
        model = self.create_model(X_copy,Y_copy, random_state=random_state, learning_rate=learning_rate)
        
        # early stopping to avoid overtraining
        earlyStopping = callbacks.EarlyStopping(monitor='val_loss', 
                                                patience=patience,verbose=self.verbose, 
                                                mode='auto')
    
        trn_desc = model.fit(X_copy[trn_id,:], Y_copy[trn_id],
                             epochs=epochs,
                             batch_size=batch_size,
                             callbacks=[earlyStopping], 
                             verbose=self.verbose,
                             validation_data=(X_copy[val_id,:],
                                              Y_copy[val_id]),
                            )
        self.model = model
        self.trn_history = trn_desc
        self.trained = True
    def predict(self, data):
        return self.model.predict(data)
    def save(self, file_path):
        with open(file_path,'wb') as file_handler:
            joblib.dump([self.n_hidden_neurons, self.model,
                        self.trn_history, self.trained], file_handler)
    def load(self, file_path):
        with open(file_path,'rb') as file_handler:
            [self.n_hidden_neurons, self.model, self.trn_history, self.trained]= joblib.load(file_handler)
    def model_with_no_output_layer(self):
        buffer_model = tf.keras.Sequential()    
        # add a input to isolate the input of NN model
        buffer_model.add(tf.keras.Input(shape=(model.model.layers[0].get_weights()[0].shape[0],)))
        # add a non-linear single neuron layer
        hidden_layer = layers.Dense(units=model.model.layers[0].get_weights()[1].shape[0],
                                    activation='tanh')
        buffer_model.add(hidden_layer)    
        output_layer = layers.Dense(units=1,activation='tanh')
    
        for idx, layer in enumerate(buffer_model.layers):
            layer.set_weights(model.model.layers[idx].get_weights())
        return buffer_model
    def predict_one_layer_before_output(self, data):
        buffer_model = self.model_with_no_output_layer()
        return buffer_model.predict(data)

In [33]:
# for kFolds CV

from sklearn.metrics import accuracy_score
from src.functions.AuxiliarFunctions import *


print(get_train_description(df_config, train_id))


if True: # remova quando tiver segurança no treinamento
    print('Training Starting')
    # data
    model_type = 'MLPNeuralNetwork'
    data = df_train.drop(columns=['target']).copy(deep=True)
    print('Data shape: (%i, %i)'%(data.shape[0], data.shape[1]))
    
    trgt = df_train['target'].copy(deep=True).values
    print('Trgt shape: %i'%(trgt.shape[0]))
    
    hidden_neurons = list(get_list_of_hidden_neurons(df_config['model_neurons'][train_id]))
    
    with open(df_config['model_status'][train_id],'rb') as file_handler:
        [model_status] = pickle.load(file_handler)
    
    
    n_folds = df_config['cv_folds'][train_id]
    for ifold in range(1):
        print('Training %i fold of %i folds'%(ifold+1, n_folds))
        # pipeline
        pipeline_name = '%s_CV_fold_%i_of_%i_cv_pipe.pkl'%(df_config['hash_id'][train_id],
                                                           ifold, n_folds)
        
        pipeline_path = df_config['pipeline_path'][train_id]
        
        with open(os.path.join(pipeline_path,pipeline_name),'rb') as file_handler:
            pipe = joblib.load(file_handler)
        trn_data = pipe.transform(data)
        trn_trgt = tf.keras.utils.to_categorical(trgt, num_classes=len(np.unique(trgt)))
        
        for idx, ineuron in enumerate(hidden_neurons):
            print('Training for %i neuron in'%(ineuron),hidden_neurons)
            
            for iinit in range(df_config['model_inits'][train_id]):
                print('Training for %i init in %i inits'%(iinit+1, df_config['model_inits'][train_id]))
                model_name = '%s_%s_%i_fold_%i_neuron_%i_init_model.pkl'%(df_config['hash_id'][train_id],
                                                                          model_type,ifold, ineuron, iinit)
                model_path = df_config['model_path'][train_id]
                #print(os.path.join(model_path, model_name))
                if os.path.exists(os.path.join(model_path, model_name)):
                    print('Modelo existente')
                    continue
                else:
                    print('Modelo não existe\n\n')
                    model = MLPModel(n_hidden_neurons=ineuron,verbose=2)
                    model.fit(trn_data, trn_trgt, trn_id=trn_idx, val_id=val_idx, 
                              epochs=10, random_state=iinit, learning_rate=0.2)
                    #model.save(os.path.join(model_path, model_name))




Toy Data Classification Training Process
Processing ../data/1518950927628997345_train_data.csv
Hidden Neurons: 1
CV Folds: 5
Inits: 1

Training Starting
Data shape: (10000, 20)
Trgt shape: 10000
Training 1 fold of 5 folds
Training for 1 neuron in [1]
Training for 1 init in 1 inits
Modelo não existe




TypeError: Unexpected keyword argument passed to optimizer: amsgrad. Allowed kwargs are {'global_clipnorm', 'lr', 'clipnorm', 'decay', 'clipvalue'}.

In [13]:
trn_data.shape

(10000, 20)